# Stage 3 : Entity matching

###  Read in the input tables A, B

In [1]:
import py_entitymatching as em

# Load data files
A = em.read_csv_metadata('./TableA.csv', key='ID')
B = em.read_csv_metadata('./TableB.csv', key='ID')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [2]:
A.head()

,ID,title,author,isbn,publisher,edition,pages,dimension
0,1,Cry Wolf (Alpha and Omega Series #1),Patricia Briggs,9780441016150,Penguin Publishing Group,NaN,320.0,4.10(w) x 6.70(h) x 1.10(d)
1,2,DK Braille: Animals,Dorling Kindersley Publishing Staff,9781465436115,DK,Large Print,32.0,8.60(w) x 10.70(h) x 0.90(d)
2,3,An American Marriage (Oprah's Book Club),Tayari Jones,9781616208776,Algonquin Books of Chapel Hill,NaN,NaN,5.90(w) x 9.10(h) x 1.30(d)
3,4,Big Stone Gap (Movie Tie-in Edition),Adriana Trigiani,9781101967447,Random House Publishing Group,Media Tie,320.0,7.90(w) x 5.10(h) x 0.80(d)
4,5,Death of a Double Dipper,Angela Pepper,9781546502661,CreateSpace Publishing,NaN,390.0,5.00(w) x 8.00(h) x 0.80(d)


In [3]:
B.head()

,ID,title,author,isbn,publisher,edition,pages,dimension
0,1,Elements Books I-XIII,Euclid,9780760763124,Barnes & Noble,Unknown,NaN,NaN
1,2,Blessed Anastacia,John Burdick,9780415912594,Routledge,Unknown,246 pages,6.22 x 9.18 x 0.77 inches
2,3,The Neighbor,Lisa Gardner,9780553591903,Bantam,Unknown,483 pages,4.00 x 7.00 x 1.50 inches
3,4,Strain The Volume 5 The Night Eternal,Guillermo Del Torro,9781616556389,Dark Horse Books,Unknown,NaN,NaN
4,5,The Edge,Scott Wittenburg,9781411686601,Lulu Press,Unknown,196 pages,6.00 x 9.00 x 0.45 inches


In [4]:
print('A: ', len(A), '\nB: ',len(B))

A:  3645 
B:  3632


### Apply the blocker

In [5]:
# Create overlap blocker
ob = em.OverlapBlocker()
# Block rule 1: titles share more than three words, split by whitespaces
C = ob.block_tables(A, B, 'title', 'title', rem_stop_words = True, overlap_size = 3, 
	l_output_attrs=['title', 'author', 'isbn', 'publisher', 'edition', 'dimension'], 
	r_output_attrs=['title', 'author', 'isbn', 'publisher', 'edition', 'dimension'] )
# Block rule 2: authors share more than two words, split by whitespaces
D = ob.block_candset(C, 'author', 'author', rem_stop_words = True, overlap_size = 2)
print('D: ', len(D))

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [##############################] 100% | ETA: 00:00:00

D:  1889



Total time elapsed: 00:00:00


In [6]:
# Debugging the blocker
dbg = em.debug_blocker(D, A, B, output_size=200)
dbg.head()

,_id,ltable_ID,rtable_ID,ltable_title,ltable_author,ltable_publisher,ltable_edition,ltable_pages,ltable_dimension,rtable_title,rtable_author,rtable_publisher,rtable_edition,rtable_pages,rtable_dimension
0,0,276,644,The Pilgrim's Progress Part One,John Bunyan,CreateSpace Publishing,NaN,102.0,6.00(w) x 9.00(h) x 0.21(d),The Pilgrim's Progress,John Bunyan,CreateSpace Independent Publishing Platform,Unknown,NaN,NaN
1,1,1216,2026,How to Write a Blog,Inklyo,CreateSpace Publishing,NaN,136.0,6.00(w) x 9.00(h) x 0.30(d),How to Write a Blog,Scribendi,CreateSpace Independent Publishing Platform,Unknown,NaN,NaN
2,2,3099,2576,Alexander Hamilton,Charles A. Conant,CreateSpace Publishing,NaN,38.0,6.00(w) x 1.25(h) x 9.00(d),Alexander Hamilton (Spanish Edition),Charles A. Conant,CreateSpace Independent Publishing Platform,Unknown,NaN,NaN
3,3,276,3482,The Pilgrim's Progress Part One,John Bunyan,CreateSpace Publishing,NaN,102.0,6.00(w) x 9.00(h) x 0.21(d),The Pilgrim's Progress,John Bunyan,CreateSpace Independent Publishing Platform,Unknown,NaN,NaN
4,4,222,1057,Child of the Ghosts,Jonathan Moeller,CreateSpace Publishing,NaN,288.0,6.00(w) x 9.00(h) x 0.60(d),Blade of the Ghosts,Jonathan Moeller,CreateSpace Independent Publishing Platform,Unknown,NaN,NaN


### Do sampling and labeling

In [7]:
# Sample candidate set of size 300
S = em.sample_table(D, 300)

In [8]:
#G = em.label_table(S, label_column_name = 'gold_labels')

The above command could open a GUI for us to update label. Also, we could do the labeling offine and read the labeled data using the command below.

In [9]:
# Gold label
G = em.read_csv_metadata('./labeled.csv', key='_id', fk_ltable='ltable_ID', fk_rtable='rtable_ID', ltable = A, rtable = B)
print('G: ',len(G))

G:  300


### Split the labeled set G into set I, J

In [10]:
# Split training set and test set
train_test = em.split_train_test(G, train_proportion=0.5)
I = train_test['train']
I['ltable_edition'] = ''
I['rtable_edition'] = ''
I['ltable_pages'] = ''
I['rtable_pages'] = ''
J = train_test['test']
J['ltable_edition'] = ''
J['rtable_edition'] = ''
J['ltable_pages'] = ''
J['rtable_pages'] = ''

Note that we made some columns empty. This was because of a discovery during debugging. See report for details.

### Create a set of ML-matchers

In [11]:
# Create ML matchers
dt = em.DTMatcher(name='DecisionTree')
svm = em.SVMMatcher(name='SVM')
rf = em.RFMatcher(name='RandomForest')
lg = em.LogRegMatcher(name='LogisticRegression')
ln = em.LinRegMatcher(name='LinearRegression')
nb = em.NBMatcher(name='NaiveBayes')

### Select the best matcher using set I

In [12]:
# Automatic feature generation
F = em.get_features_for_matching(A, B, validate_inferred_attr_types = False)
H = em.extract_feature_vecs(I, feature_table=F, attrs_after=['gold_labels'])
# Fill missing values
H.fillna(value='NaN', inplace=True)
# Select the best matcher
result = em.select_matcher([dt,rf, svm, ln, lg, nb], table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_labels'], k=5, target_attr='gold_labels', metric_to_select_matcher='f1')
print(result['cv_stats'])
best_matcher = result['selected_matcher']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


              Matcher  Average precision  Average recall  Average f1
0        DecisionTree           0.928235        0.930211    0.940602
1        RandomForest           0.975000        0.950000    0.955225
2                 SVM           0.712852        0.848985    0.765040
3    LinearRegression           0.986667        0.953100    0.971080
4  LogisticRegression           0.961569        0.956725    0.937485
5          NaiveBayes           0.942222        0.947753    0.944763


### Evaluate the best matcher Y using J

In [13]:
# Evaluate the best matcher
L = em.extract_feature_vecs(J, feature_table=F, attrs_after=['gold_labels'])
L.fillna(value='NaN', inplace=True)
best_matcher.fit(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_labels'], target_attr='gold_labels')
predictions = best_matcher.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold_labels'], target_attr='predicted_labels', append = True)
eval_result = em.eval_matches(predictions, 'gold_labels', 'predicted_labels')
print('----------Best Matcher: ', best_matcher.name, '----------')
em.print_eval_summary(eval_result)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


----------Best Matcher:  LinearRegression ----------
Precision : 98.63% (72/73)
Recall : 96.0% (72/75)
F1 : 97.3%
False positives : 1 (out of 73 positive predictions)
False negatives : 3 (out of 77 negative predictions)
